In [6]:
import spark.implicits._
import org.apache.spark.sql.types.TimestampType
val fligthsDf = spark.read.option("header","true").option("inferSchema","true").csv("../../resources/chapter4/departuredelays.csv")
//Convert unix to timestamp
val fligthsDateDf = fligthsDf.select($"date".cast(TimestampType).as("date"),$"delay",$"distance",$"origin",$"destination")

import spark.implicits._
import org.apache.spark.sql.types.TimestampType
fligthsDf: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]
fligthsDateDf: org.apache.spark.sql.DataFrame = [date: timestamp, delay: int ... 3 more fields]


In [7]:
//Create temporary view
    fligthsDateDf.createOrReplaceTempView("us_delay_flights_tbl")

In [9]:
//Do some querys over temporary view
    spark.sql("""SELECT distance, origin, destination
     FROM us_delay_flights_tbl
     WHERE distance > 1000
     ORDER BY distance DESC""").show(10)

    spark.sql("""SELECT date, delay, origin, destination
     FROM us_delay_flights_tbl
     WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
     ORDER by delay DESC""").show(10)

    spark.sql("""SELECT delay, origin, destination,
     CASE
       WHEN delay > 360 THEN 'Very Long Delays'
       WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
       WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
       WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
       WHEN delay = 0 THEN 'No Delays'
       ELSE 'Early'
     END AS Flight_Delays
     FROM us_delay_flights_tbl
     ORDER BY origin, delay DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows

+-------------------+-----+------+-----------+
|               date|delay|origin|destination|
+-------------------+-----+------+-----------+
|1970-01-26 08:35:25| 1638|   SFO|        ORD|
|1970-01-12 22:35:55|  396|   SFO|        ORD|
|1970-01-12 19:58:50|  326|   SFO|        ORD|
|1970-01-13 04:00:05|  320|   SFO|        ORD|
|1970-01-14 18:48:45|  297|   SFO|        ORD|
|1970-01-26 03:05:15|  296|   SFO|        ORD|
|1970-01-13 09:30:40|  279|   SFO|        ORD|
|1970-01-13 04:05:50|  274|   SFO|        ORD|
|1970-02-06 02:52:10|  266|   SFO|   

In [11]:
//Read files created in chapter3
spark.read.option("header","true").option("inferSchema","true").csv("../../output/scala/chapter3/fireCalls/").show(5)
spark.read.option("inferSchema","true").option("inferSchema","true").json("../../output/scala/chapter3/fireCallsDates/").show(5)
spark.read.parquet("../../output/scala/chapter3/commonCallsDates/").show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------